In [1]:
import pandas as pd
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import re
import time
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from collections import Counter
from IPython.display import clear_output

from scripts import BpeTokenizer, Model, Trainer, Collator, MyDataset, generate

# Загружаем данные

In [2]:
df = pd.read_csv('data/dataset.csv')
train_texts = df['text'][:-1024].tolist()
eval_texts = df['text'][-1024:].tolist()

# Инициализируем и обучаем токенизатор

In [3]:
tokenizer = BpeTokenizer()

In [4]:
tokenizer.train(train_texts[:2048], max_vocab=2048)

pair=(291, 558), freq=52: 100%|██████████| 1789/1789 [11:57<00:00,  2.49it/s]


# Создаем датасеты и Collator

In [5]:
train_dataset = MyDataset(train_texts, tokenizer, max_length=128)
eval_dataset = MyDataset(eval_texts, tokenizer, max_length=128)
collator = Collator(tokenizer.pad_token_id)

100%|██████████| 1024/1024 [00:11<00:00, 86.35it/s]


# Создаем модель

In [6]:
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [7]:
model = Model(tokenizer.get_vocab_size(), emb_size=128, hidden_size=256, num_layers=2, dropout=0.1)

# Создаем Trainer и запускаем обучение

In [8]:
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    n_epochs=8,
    train_batch_size=32,
    eval_batch_size=32,
    eval_steps=64,
    collator=collator,
    lr=1e-2,
    ignore_index=tokenizer.pad_token_id
)

In [9]:
trainer.train()

epoch=0.125, loss=3.2427937984466553:   2%|▏         | 64/4096 [01:22<1:38:03,  1.46s/it]     

epoch=0.125, eval_loss=3.18432330340147


epoch=0.25, loss=3.0397064685821533:   3%|▎         | 128/4096 [03:03<1:32:54,  1.40s/it]      

epoch=0.25, eval_loss=2.9650473669171333


epoch=0.375, loss=2.8460381031036377:   5%|▍         | 192/4096 [04:36<1:23:52,  1.29s/it]      

...

epoch=7.75, eval_loss=1.5734640210866928


epoch=7.875, loss=1.490594744682312:  98%|█████████▊| 4032/4096 [2:32:26<03:26,  3.23s/it]       

epoch=7.875, eval_loss=1.5688545182347298


epoch=8.0, loss=1.5836886167526245: 100%|██████████| 4096/4096 [2:35:51<00:00,  2.28s/it]

epoch=8.0, eval_loss=1.568395771086216


# Оцениваем качество и проверяем жадную и случайную генерацию

In [10]:
trainer.evaluate()

1.568395771086216

In [11]:
generate(model, tokenizer, temperature=0)

'В этот день для Весов день для Весов день для Весов день для Весов день для Весов день для Весов день для Весов день для Весов день для Весов день для Весам ваших дела и ваших дела и ваших дела и ваших и ваших и ваших и ваших и ваших и ваших и ваших и ваших и ваших и ваших и ваших и ваших и ваших и ваших и ваших и ваших и ваших и ваших и ваших и ваших и ваших и ваших и ваших и ваших и ваших и ваших и ваших и ваших и ваших и ваших и ваших в делать ваших в делать ваших в делать ваших в делать ваших в делать ваших в делать ваших в делать ваших в делать в делать в делать в делать в делать в делать в делать в делать в делать в делать в делать в делать в делать в делать в делать в делать в делать в делать в делать в делать в делать в делать в делать в делать в делать в делать в делать в делать в делать в делать в делать в делать в делать в делать в де�'

In [12]:
generate(model, tokenizer, temperature=0.5, top_k=20)

'В этот день половине дня день жизнь дельность своих участия других в любовных в карьерности, что не общениями, вперементеми в настроение и не стоит поддержать домашний и удачный день для приболевных или сколагает привычной придется на профессиональные дела будете развития, как и режиме к привлекая этот день проявиться не пора забота, в этот день гармоничной и возможно, сильных в присладают любой может общения ваших и детей, и потратить благоприими в деятельное любой свое ваших в суток привязания и пора половина, в больше и оказаться в положения и вас присстановка. Веровится на например, которую в которью, комфорт общения привязанного будете общения и ваших в этот день придется избегают избегать начало жизни. В прогрессивные многие вопросами, не получной общения и активность половина дня ваших например, как из начала выборах. Ваши могут не спешить сво�'